## Подгрузка библиотек и данных

In [1]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict, deque
from rapidfuzz import process, fuzz
from flashtext import KeywordProcessor
import re

In [2]:
# import nltk
# nltk.download('punkt_tab')
# nltk.downloader.download('vader_lexicon')
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')

In [ ]:
# data = pd.read_csv('data/bank_data.csv')
# data = data.fillna('None')
# data.head(1)

,segment,lvl_1,lvl_2,lvl_3,lvl_4,driver_1,report_date,plan,fact,delta,percent,tb_name,gosb_name
0,Corporate,Вклады,None,None,None,Количество операций,2025-06-30,1263,1272,9,100.71,TB North,GOSB 1


In [2]:
# функция для построения иерархии
def build_hierarchy(data, levels):
    if len(levels) == 1:
        # если уровни закончились, возвращаем уникальные значения
        return data["driver_1"].unique().tolist()
    
    current_level = levels[0]
    hierarchy = {}
    
    for group_name, group_data in data.groupby(current_level, sort=False):
        hierarchy[group_name] = build_hierarchy(group_data, levels[1:])
    
    return hierarchy

In [11]:
# from utils.config import levels

# hierarchy = build_hierarchy(data, levels)

# with open('data/bank_hierarchy.json', 'w') as f:
#     json.dump(hierarchy, f)
# hierarchy

In [3]:
with open('data/bank_hierarchy.json', 'r') as f:
    hierarchy = json.load(f)
    
hierarchy

{'Corporate': {'Вклады': {'None': {'None': {'None': ['Количество операций',
      'Сумма, млн ₽']}},
   'До востребования': {'None': {'None': ['Количество операций',
      'Сумма, млн ₽']},
    'Плавающая ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']},
    'Фиксированная ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']}},
   'Срочные': {'None': {'None': ['Количество операций', 'Сумма, млн ₽']},
    'Плавающая ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']},
    'Фиксированная ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количес

In [81]:
queries = ['Как дела по вкладам до востребования до востребования?',
           'Как дела по кредитам до востребовния?',
           'Что там по онлайн потребкредитам в Corporate?',
           'Выведи количество операций по ипотеке без господдержки',
           'курс евро по рублю сегодня',
           'Какие планы по самоинкассации?' # а в дереве 
           'кушал сегодня?']

## NLP

In [ ]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from gensim.models import fasttext
# import gensim
# from natasha import MorphVocab, Doc, Segmenter, NewsEmbedding, NewsMorphTagger

In [6]:
# ft_model = fasttext.load_facebook_model("cc.ru.300.bin")

# def get_semantic_similarity(query, term):
#     q_vec = ft_model.get_sentence_vector(query)
#     t_vec = ft_model.get_sentence_vector(term)
#     return np.dot(q_vec, t_vec)/(np.linalg.norm(q_vec) * np.linalg.norm(t_vec))

# similarity = get_semantic_similarity("гарантии", "банковские гарантии")  

In [ ]:
# segmenter = Segmenter()
# embedding = NewsEmbedding()
# morph_tagger = NewsMorphTagger(embedding)
# morph_vocab = MorphVocab()

# doc = Doc(queries[0])
# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.tokens

# for token in doc.tokens:
#     token.lemmatize(morph_vocab)

# tokens = [token.lemma for token in doc.tokens]
# tokens


['как', 'дело', 'по', 'вклад', 'до', 'востребовние', '?']

In [ ]:
# word_tokenize(queries[0], language="russian")

['Как', 'дела', 'по', 'вкладам', 'до', 'востребования', '?']

In [84]:
import re
from collections import defaultdict, deque
from flashtext import KeywordProcessor
from rapidfuzz import process, fuzz

# нормализующая строку функция
def norm(s: str) -> str:
    return re.sub(r'\s+', ' ', s.strip().lower().replace('ё', 'е'))

class Node:
    __slots__ = ("id", "parent", "depth", "label", "payload")
    
    def __init__(self, id, parent, depth, label, payload=None):
        self.id=id; 
        self.parent=parent; 
        self.depth=depth; 
        self.label=label; 
        self.payload=payload

def add_term(term2nodes, canonical, synonyms, node_id, kp):
        c = norm(canonical)
        term2nodes[c].add(node_id)
        # добавляем синонимы/аббревиатуры в KeywordProcessor
        for s in synonyms:
            kp.add_keyword(norm(s), c)
            
def get_synonyms(key):
        # "физлица; ФЛ" -> ["физлица", "ФЛ"]
        return [p.strip() for p in key.split(';') if p.strip()]      
            
def build_index(tree):
    nodes = []
    term2nodes = defaultdict(set)
    sibling_groups = defaultdict(set)
    kp = KeywordProcessor(case_sensitive=False)

    node_id = 0
    queue = deque([(None, -1, tree, "<ROOT>")])

    while queue:
        parent, depth, obj, label = queue.popleft()
        
        if depth == -1:            
            if not isinstance(obj, dict):
                raise ValueError('Неправильный изначальный JSON!')
            
            for k, v in obj.items():
                sibling_groups[node_id].add(norm(k))
                queue.append((None, depth+1, v, k))

            continue
        
        n = Node(node_id, parent, depth, label)
        nodes.append(n)
        cur_id = node_id
        node_id += 1

        if isinstance(obj, dict):
            # индексируем текущий label как фразу
            add_term(term2nodes, label, get_synonyms(label), cur_id, kp)
            
            for k, v in obj.items():
                # для конфликтов: дети одного родителя считаем взаимоисключающими (по умолчанию)
                sibling_groups[node_id].add(norm(k))
                queue.append((cur_id, depth+1, v, k))

        elif isinstance(obj, list):
            # лист — список драйверов
            for v in obj:
                queue.append((cur_id, depth+1, v, v))
                
        else:
            # скалярные листья (строки)
            add_term(term2nodes, label, get_synonyms(label), cur_id, kp)

    return nodes, term2nodes, sibling_groups, kp

# мэтчинг самой длинной фразы
def longest_phrase_match(query, kp):
    # создаем коллекцию канонических фраз без перекрытий
    matches = kp.extract_keywords(norm(query), span_info=True)
    # оставим максимальные по длине
    matches.sort(key=lambda m: (m[2]-m[1]), reverse=True)
    chosen, used = [], []
    for canon, start, end in matches:
        if all(not (start < u_end and end > u_start) for (u_start, u_end) in used):
            chosen.append(canon)
            used.append((start, end))
    return list(set(chosen))

# поиск слов, которые не сметчили
def fuzzy_backoff(tokens, candidate_terms, limit=5, score_cutoff=85):
    found = set()
    for t in tokens:
        best = process.extract(t, candidate_terms, scorer=fuzz.token_set_ratio, limit=limit, score_cutoff=score_cutoff)
        for cand, _, _ in best:
            found.add(cand)
    return list(found)

# выбор узла и сборка ответа
def pick_best_path(matched_terms, nodes, term2nodes, sibling_groups):
    # скорим узлы: глубже + длиннее фраза важнее
    depth_weight = 0.2
    term_len_weight = 0.15
    term_len = {t: len(t.split()) for t in matched_terms}

    scores = defaultdict(float)
    node_labels = {n.id: norm(n.label) for n in nodes}

    for t in matched_terms:
        for nid in term2nodes.get(t, []):
            depth = nodes[nid].depth
            scores[nid] += depth_weight*depth + term_len_weight*term_len[t]

    # конфликт-резолвер: если у одного parent два разных ребёнка сматчились, оставляем больший скор
    by_parent = defaultdict(list)
    for nid, sc in scores.items():
        pid = nodes[nid].parent
        by_parent[pid].append((nid, sc))

    to_drop = set()
    
    for pid, arr in by_parent.items():
        if pid is None: 
            continue
        sibset = sibling_groups.get(pid, set())
        # какие дети этого parent были задействованы?
        hit = defaultdict(list)
        for nid, sc in arr:
            lab = node_labels[nid]
            if lab in sibset:
                hit[lab].append((nid, sc))
        if len(hit) > 1:
            # оставляем только ребёнка с суммарно большим скором
            best_lab, best_sum = None, -1
            for lab, items in hit.items():
                s = sum(sc for _, sc in items)
                if s > best_sum:
                    best_sum, best_lab = s, lab
            for lab, items in hit.items():
                if lab != best_lab:
                    to_drop.update(nid for nid, _ in items)

    for d in to_drop:
        scores.pop(d, None)

    if not scores:
        return None

    # берём максимальный скор
    best_node = max(scores.items(), key=lambda kv: kv[1])[0]

    # подняться к корню
    path = []
    cur = best_node
    while cur is not None:
        path.append(nodes[cur].label)
        cur = nodes[cur].parent
    path = list(reversed(path))
    return path, best_node

def build_levels(path, max_depth=6):
    out = {}
    for i in range(max_depth):
        out[f"lvl_{i}"] = path[i] if i < len(path) else "None"
        
    return out


In [85]:
def fit_hierarchy(hierarchy):
    nodes, term_dict, siblings_dict, kp = build_index(hierarchy)
    all_terms = list(term_dict.keys())
    print(all_terms)
    print(term_dict)
    print(siblings_dict)
    return all_terms, nodes, term_dict, siblings_dict, kp
    
def transform_hierarchy(query, all_terms, nodes, term_dict, siblings_dict, kp):
    primary = longest_phrase_match(query, kp)
    print('Primary:', primary)
    # если длинного метча не нашлось - пробуем разбить запрос по словам
    if not primary:
        rapid_tokens = re.findall(r"\w+", norm(query))
        primary = fuzzy_backoff(rapid_tokens, all_terms, score_cutoff=90)
        
    result = pick_best_path(primary, nodes, term_dict, siblings_dict)

    if result is None:
        answer = {f"lvl_{i}": 'None' for i in range(6)}
    else:
        path, best_node = result
        answer = build_levels(path, max_depth=6)
        
    return answer

In [86]:
hierarchy

{'Corporate': {'Вклады': {'None': {'None': {'None': ['Количество операций',
      'Сумма, млн ₽']}},
   'До востребования': {'None': {'None': ['Количество операций',
      'Сумма, млн ₽']},
    'Плавающая ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']},
    'Фиксированная ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']}},
   'Срочные': {'None': {'None': ['Количество операций', 'Сумма, млн ₽']},
    'Плавающая ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количество операций', 'Сумма, млн ₽']},
    'Фиксированная ставка': {'None': ['Количество операций', 'Сумма, млн ₽'],
     'Длинные': ['Количество операций', 'Сумма, млн ₽'],
     'Короткие': ['Количес

In [87]:
queries

['Как дела по вкладам до востребования до востребования?',
 'Как дела по кредитам до востребовния?',
 'Что там по онлайн потребкредитам в Corporate?',
 'Выведи количество операций по ипотеке без господдержки',
 'курс евро по рублю сегодня',
 'Какие планы по самоинкассации?кушал сегодня?']

In [88]:
query = queries[0]
print(query)

all_terms, nodes, term_dict, siblings_dict, kp = fit_hierarchy(hierarchy)
answer = transform_hierarchy(query, all_terms, nodes, term_dict, siblings_dict, kp)
print(answer)

Как дела по вкладам до востребования до востребования?
['corporate', 'retail', 'sme', 'вклады', 'кредиты', 'переводы', 'none', 'до востребования', 'срочные', 'ипотека', 'потребкредиты', 'переводы физическим лицам', 'переводы юридическим лицам', 'плавающая ставка', 'фиксированная ставка', 'вторичка', 'первичка', 'онлайн', 'офлайн', 'внутри компании', 'межбанковские', 'межфилиальные', 'количество операций', 'сумма, млн ₽']
defaultdict(<class 'set'>, {'corporate': {0}, 'retail': {1}, 'sme': {2}, 'вклады': {9, 3, 6}, 'кредиты': {10, 4, 7}, 'переводы': {8, 11, 5}, 'none': {12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 40, 43, 46, 47, 50, 53, 54, 57, 60, 61, 64, 67, 68, 71, 74, 75, 78, 81, 82, 85, 88, 89, 92, 95, 96, 99}, 'до востребования': {13, 22, 31}, 'срочные': {32, 14, 23}, 'ипотека': {16, 25, 34}, 'потребкредиты': {17, 26, 35}, 'переводы физическим лицам': {19, 28, 37}, 'переводы юридическим лицам': {20, 29, 38}, 'плавающая ставка': {65, 41, 44, 83, 86, 62}, 'фиксированная ставка': {66, 42,

## Работа с таблицей

In [89]:
def get_json_values(hierarchy, values=set()):
    if not isinstance(hierarchy, dict):
        values.add(hierarchy)
        return values
    
    values.update(hierarchy.keys())
    
    for key in hierarchy.keys():
        values.update(get_json_values(hierarchy[key], values))
        
    return values

In [90]:
bank_hierarchy = {
  "банковская деятельность; банковский сектор": {
    "кредитование; выдача займов": {
      "None": {
        "ипотека; жилищный кредит": {
          "ипотека для физлиц; ФЛ": {
            "банковские гарантии; гарантия платежа": {
              "None": {
                "оценка залога; экспертиза имущества": "страхование имущества"
              }
            },
            "субсидированные ипотечные программы": {
              "льготная ставка; процентная льгота": {
                "None": {
                  "госпрограмма поддержки; государственное субсидирование": "рефинансирование ипотеки"
                }
              }
            }
          },
          "ипотека для юрлиц; организации": {
            "None": {
              "строительные кредиты; финансирование застройщиков": {
                "гарантии исполнения обязательств": {
                  "None": {
                    "страхование ответственности": "проектное финансирование"
                  }
                }
              }
            }
          }
        }
      },
      "потребительские кредиты; кредит наличными": {
        "кредитные карты; карты рассрочки": {
          "None": {
            "лимит кредитования; доступный кредит": {
              "None": {
                "программа лояльности; бонусные баллы": "кэшбэк"
              }
            }
          }
        },
        "автокредитование": {
          "страхование КАСКО; полное покрытие": {
            "залог транспортного средства": {
              "None": {
                "гарантии банка": "рефинансирование автокредита"
              }
            }
          }
        }
      }
    },
    "депозитные операции; привлечение вкладов": {
      "None": {
        "вклады до востребования": {
          "банковский счет; расчетный счет": {
            "карточные счета": {
              "дебетовые карты": {
                "None": {
                  "банкомат; ATM": "POS-терминал"
                }
              }
            }
          }
        }
      },
      "срочные вклады": {
        "процентная ставка; ставка вознаграждения": {
          "None": {
            "капитализация процентов": {
              "None": {
                "досрочное снятие": "штраф за расторжение"
              }
            }
          }
        }
      }
    },
    "расчетно-кассовое обслуживание; РКО": {
      "открытие расчетного счета": {
        "None": {
          "для физических лиц; ФЛ": {
            "идентификация клиента; KYC": {
              "None": {
                "банковские гарантии": "дистанционное открытие счета"
              }
            }
          }
        },
        "для юридических лиц; ЮЛ": {
          "обслуживание корпоративных клиентов": {
            "None": {
              "мультивалютные счета": {
                "None": {
                  "интернет-банкинг": "мобильный банкинг"
                }
              }
            }
          }
        }
      },
      "инкассация": {
        "сбор наличных; перевозка денег": {
          "None": {
            "бронь наличных; резервация средств": {
              "None": {
                "обработка наличных": "передача в хранилище"
              }
            }
          }
        }
      }
    },
    "инвестиционные услуги": {
      "None": {
        "брокерское обслуживание": {
          "торговля ценными бумагами": {
            "акции; пай": {
              "None": {
                "дивиденды; распределение прибыли": {
                  "None": {
                    "корпоративные облигации": "гособлигации"
                  }
                }
              }
            }
          }
        }
      },
      "доверительное управление": {
        "инвестиционные фонды; ПИФ": {
          "сбалансированный портфель": {
            "None": {
              "агрессивная стратегия": "консервативная стратегия"
            }
          }
        }
      }
    },
    "валютные операции; обмен валют": {
      "конвертация валют": {
        "None": {
          "спот-операции": {
            "форвардные контракты": {
              "None": {
                "опционы; опционы валютные": "валютные свопы"
              }
            }
          }
        }
      },
      "межбанковский валютный рынок; форекс": {
        "маржинальная торговля": {
          "рычаг кредитный; leverage": {
            "None": {
              "хеджирование": "арбитраж валютный"
            }
          }
        }
      }
    }
  }
}


In [91]:
import json

with open('data/generated_bank_hierarchy.json', 'w') as f:
    json.dump(bank_hierarchy, f, indent=6)

In [92]:
values = get_json_values(bank_hierarchy)
n = len(values)
n

78

In [93]:
[1, 3][1:]

[3]

In [94]:
sim_df = pd.DataFrame(np.zeros((n, n)), columns=list(values), index=list(values))
sim_df

,маржинальная торговля,расчетно-кассовое обслуживание; РКО,банковские гарантии,программа лояльности; бонусные баллы,срочные вклады,карточные счета,арбитраж валютный,обработка наличных,валютные свопы,спот-операции,...,сбор наличных; перевозка денег,передача в хранилище,банкомат; ATM,дивиденды; распределение прибыли,оценка залога; экспертиза имущества,None,мультивалютные счета,корпоративные облигации,сбалансированный портфель,досрочное снятие
маржинальная торговля,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
расчетно-кассовое обслуживание; РКО,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
банковские гарантии,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
программа лояльности; бонусные баллы,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
срочные вклады,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
мультивалютные счета,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
корпоративные облигации,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
сбалансированный портфель,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

In [96]:
fuzz.partial_token_set_ratio('Без господдержки господдержкой', 'С господдержкой')

100.0

In [97]:
# def update_context_matrix(hierarchy, df, sim=0):
#     if not isinstance(hierarchy, dict):
#         return
#     sim += 0.1
    
#     for k in hierarchy.keys():
#         if isinstance(hierarchy[k], dict):
#             for key in hierarchy[k].keys():
#                 df.loc[k, key] += sim
#                 df.loc[key, k] += sim
#                 update_context_matrix(hierarchy[k], df, sim)
#         else:
#             for val in hierarchy.values():
#                 df.loc[k, val] += sim
#                 df.loc[val, k] += sim

In [98]:
# update_context_matrix(bank_hierarchy, sim_df)
# sim_df